In [1]:
# basic torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim

# os related
import os

# segmentation head model
from segmentation_head import CustomSegHead
# dataset class
from pet_dataset_class import PreprocessedPetDataset
# dataloaders
from create_dataloaders import get_pet_dataloaders
# loss and metrics
from loss_and_metrics_seg import * # idk what to import here tbh. Need to look into it
# trainer
from trainer import trainer

# data plotting
from data_plotting import plot_random_images_and_trimaps_2


In [ ]:
check_seg_head_model = #Seg_head_new(...) # define your own similar segmentation head
#CustomSegHead(hidden_dim=768, num_classes=3, patch_size=16, image_size=224) 
check_seg_head_model.num_trainable_params 

2655747

In [3]:
# get path of image and mask files
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # __file__ is not defined (e.g. in Jupyter notebook or interactive sessions apparently), fallback to cwd
    base_dir = os.getcwd()

# Suppose your dataset is in a folder named 'data' inside the project root:
data_dir = os.path.join(base_dir, 'data_oxford_iiit')

# # Then you can define image and trimap paths relative to that
image_folder = os.path.join(data_dir, 'resized_images')
trimap_folder = os.path.join(data_dir, 'resized_masks')

In [5]:
# create dataloaders
train_dl, val_dl, test_dl = get_pet_dataloaders(
    image_folder=image_folder,
    mask_folder=trimap_folder,
    DatasetClass=PreprocessedPetDataset,
    all_data=False,
    num_datapoints=80, # right now I am just testing whether the things would work or not. I will intentionally overtrain it.
    batch_size=24
) 

[INFO] Using only 80 datapoints out of 7390 total files.
train_size: 64, val_size: 8 and test_size: 8


In [11]:
# define the input parameters for the trainer
trainer_segmentation_params = {
    "model": check_seg_head_model , 
    "optimizer": torch.optim.Adam(check_seg_head_model.parameters(), lr=1e-4),
    "lr": 1e-4,
    "criterion": log_cosh_dice_loss,  # or log_cosh_dice_loss, whichever you want to use
    "num_epoch": 10,
    "dataloaders": {
        "train": train_dl,  # replace with your actual DataLoader
        "val": val_dl       # replace with your actual DataLoader
    },
    "use_trap_scheduler": False,             # or your scheduler instance if you use one
    "device": "cpu",#"cuda" if torch.cuda.is_available() else "cpu",
    #"model_kwargs": {},            # add any extra forward() kwargs if needed
    "criterion_kwargs": {
        "num_classes": 3,
        "epsilon": 1e-6,
        # "return_metrics": False    # usually False for training, True for validation if you want metrics ## WE DO NOT NEED THIS HERE
    }
}

In [19]:
for idx, data in enumerate(train_dl):
    datas = data[0]
    remapped = data[1]
    print("img_tensor shape:", datas.shape)
    print("remapped_tensor shape:",remapped.shape)
    break

img_tensor shape: torch.Size([24, 3, 224, 224])
remapped_tensor shape: torch.Size([24, 224, 224])


In [12]:
# instantiate the trainer
trainer_seg_head =  trainer(**trainer_segmentation_params)

In [14]:
trainer_seg_head.train() # so this is inde

Epochs:   0%|          | 0/10 [00:33<?, ?it/s]


ValueError: too many values to unpack (expected 3)